In [1]:
!pip install -U datasets
!pip install pandas
!pip install sacrebleu

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec[http]<=2023.10.0,>=2023.1.0 from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: datasets


In [2]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 624.9 kB/s eta 0:00:00


In [3]:
from bert_score import score

In [4]:
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
import torch
from sacrebleu import corpus_bleu

In [5]:
# Load the model and tokenizer once
model_name = 'Helsinki-NLP/opus-mt-en-fr'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [6]:
# Move the model to GPU if available
if torch.cuda.is_available():
    model = model.to('cuda')

def translate_to_french(text):
    encoded_text = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)
    if torch.cuda.is_available():
        encoded_text = encoded_text.to('cuda')
    translated = model.generate(encoded_text)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

def batch_translate_to_french(sentences):
    tokens_fr = tokenizer(sentences.tolist(), return_tensors="pt", padding=True, truncation=True)
    if torch.cuda.is_available():
        tokens_fr = {k: v.to('cuda') for k, v in tokens_fr.items()}
    translated_fr = model.generate(**tokens_fr)
    return tokenizer.batch_decode(translated_fr, skip_special_tokens=True)

In [7]:
# Test the translation function
english_text = "I love you"
french_text = translate_to_french(english_text)
print(english_text)
print(french_text)

I love you
Je t'aime.


In [8]:
# Load the dataset
df = pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv', nrows=1000)
df['French_Translation'] = ''

In [9]:
# Translate the dataset in batches
batch_size = 48
for i in range(0, len(df), batch_size):
    english_sentences = df.loc[i:i+batch_size-1, 'en']
    translated_fr_batch = batch_translate_to_french(english_sentences)
    df.loc[i:i+batch_size-1, 'French_Translation'] = translated_fr_batch

In [10]:
df

,en,fr,French_Translation
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...,Changing Lives (en anglais seulement) Changing...
1,Site map,Plan du site,Plan du site
2,Feedback,Rétroaction,Commentaires
3,Credits,Crédits,Crédits ouverts
4,Français,English,Français
...,...,...,...
995,A nova is a star that absorbs matter from a ne...,La matière absorbée finit par réchauffer l'éto...,Une nova est une étoile qui absorbe la matière...
996,It is a rare and spectacular event.,Il s'agit d'un phénomène rare et plutôt specta...,C'est un événement rare et spectaculaire.
997,"In 1977, he became the French editor of the Na...","En 1977, il devient l'éditeur francophone du N...","En 1977, il devient rédacteur en chef français..."
998,"In 1978, Lemay began the daunting task of asse...","En 1978, il entreprend la tâche colossale de r...","En 1978, Lemay commence la tâche redoutable d'..."


In [11]:
# Calculate the BLEU score
reference_translations = df['fr'].tolist()
candidate_translations = df['French_Translation'].tolist()
bleu_score = corpus_bleu(candidate_translations, [reference_translations])
print(f"SacreBLEU Score: {bleu_score.score}")

SacreBLEU Score: 40.78508370260658


In [12]:
# Calculate the BERTScore
references = df['fr'].tolist()  # column 'fr' with reference translations
hypotheses = df['French_Translation'].tolist()  # 'French_Translation' : column with model translations 
P, R, F1 = score(hypotheses, references, lang='fr', verbose=True)
print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall: {R.mean().item():.4f}")
print(f"BERTScore F1: {F1.mean().item():.4f}")

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

calculating scores...
computing bert embedding.


  0%|          | 0/31 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 5.81 seconds, 172.26 sentences/sec
BERTScore Precision: 0.8966
BERTScore Recall: 0.8943
BERTScore F1: 0.8952
